In [ ]:
from collections import Counter
from wordcloud import WordCloud

import pandas as pd
import numpy as np
import pathlib
import re
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt

import spacy

# Sourcing Data

In [ ]:
pathlib.Path( pathlib.os.getcwd(),'data')

## Create Unified Dataframe

In [3]:
# Create Dataframe
columns =['date', 'name', 'user_id', 'verified', 'product', 'review_comment','stars', 'review']
df = pd.DataFrame(columns=columns)
# Insert JSON files into unified dataframe
for path in pathlib.Path(pathlib.os.getcwd(),'data').glob('*.json'):
    temp = pd.read_json(path)
    df = df.append(temp)

In [4]:
df.review.str.len().describe()

count    24732.000000
mean       496.538937
std        407.003654
min          2.000000
25%        177.000000
50%        435.000000
75%        664.000000
max       8744.000000
Name: review, dtype: float64

# Cleaning Data

In [5]:
# Looking number of Comments by Organization
mask = df.name.eq('WalletHub')
print('Content from site: {}'.format(df[mask].shape[0]))
df[mask].sort_values('date').tail()

Content from site: 70


,date,name,user_id,verified,product,review_comment,stars,review
2,2016-09-28,WalletHub,WalletHub,False,TD Bank Credit Cards,False,4,The TD Bank Cash Visa Card is an average offer...
1,2020-04-20,WalletHub,WalletHub,False,None,True,4,TD Bank Personal Loan ReviewTD Bank offers ine...
0,2020-04-20,WalletHub,WalletHub,False,PNC Personal Loans,False,5,PNC Personal Loan ReviewPNC personal loans hav...
1,2020-05-04,WalletHub,WalletHub,False,None,True,4,Citibank Personal Loan ReviewCitibank personal...
0,2021-08-11,WalletHub,WalletHub,False,Wells Fargo Personal Loans,False,4,\nWells Fargo Personal Loan Review\n \nWells ...


In [6]:
# Look at non comments
mask = df.review_comment.eq(True)
print('Number of Replies: {}'.format(df[mask].shape[0]))
df[mask].sort_values('date').tail()

Number of Replies: 1143


,date,name,user_id,verified,product,review_comment,stars,review
1406,2021-08-31,lisa,LisaInCali,False,None,True,0,Amex is the same....reports 3 mos after openin...
78,2021-08-31,Thomas Garrison,tagarrisonllc,False,None,True,0,"Don't be surprised if they, without any commun..."
18,2021-09-01,Rocky Grand,ullas_idol,False,None,True,1,Beware of Horrible disputed transactions suppo...
306,2021-09-02,whuser16449284,whuser16449284,False,None,True,0,Mine has no annual fee. It is $0.00
306,2021-09-02,arniejonassen,arniejonassen,False,None,True,0,"Same here. It said I had a $5,000 credit limi..."


## Removing non-relevant samples

In [7]:
"""Removing comments and or content posted by Site"""
print('Size before {}'.format(df.shape[0]))
df = df[df.name.ne('WalletHub') & df.review_comment.ne(True)].reset_index(drop=True)
print('Size after {}'.format(df.shape[0]))

Size before 24732
Size after 23583


## Assigning Bank Labels

In [8]:
banks = ['Wells Fargo', 'Bank of America', 'Citibank', 'Chase', 
         'PNC', 'TD Bank', 'Capital One', 'U.S. Bank']

for bank in banks:
    df.loc[df['product'].str.contains(bank), 'bank'] = bank
df.head()

,date,name,user_id,verified,product,review_comment,stars,review,bank
0,2021-09-08,katelyn,katelyn_leifert,True,PNC Credit Cards,False,5,The PNC cash rewards card is a great first cre...,PNC
1,2021-09-07,Jessica K,jessicak8652,False,PNC Mortgages,False,1,Stay as far away from this lender as possible....,PNC
2,2021-09-06,Doris,dorish_33,True,PNC Credit Cards,False,5,I like everything about it don't want to chang...,PNC
3,2021-09-02,Virgil,virgilw_7,True,PNC Credit Cards,False,5,Good card....0 percent interest for first year...,PNC
4,2021-09-02,shloymie,shloymie,False,PNC Business Services,False,2,Local branch very Hard to reach by phone. Gene...,PNC


In [9]:
# Look at samples per bank
df.bank.value_counts()

Capital One        8585
Chase              4025
Citibank           3419
Bank of America    2797
Wells Fargo        1838
U.S. Bank          1054
PNC                 983
TD Bank             882
Name: bank, dtype: int64

## Cleaning Product Categories

In [10]:
# Replacing names from products
pattern = re.compile(" |".join(banks) + " ")

df['product'] = df['product'].apply(lambda text: re.sub(pattern=pattern, repl="", string=text))

In [11]:
df['product'].value_counts()

Credit Cards         17193
Checking              3965
Car Loans              758
Savings & CDs          663
Mortgages              389
Business Services      262
Prepaid Cards          105
Personal Loans          87
Home Equity             49
Savings                 48
Student Loans           33
Investments             22
CDs                      7
CD Rates                 1
Conventional             1
Name: product, dtype: int64

In [12]:
# Checking CDs
def convert_to_product(old_product, new_product):

    # affected rows
    rows = df[df['product'].eq(old_product)].shape[0]
    rows_new = df[df['product'].eq(new_product)].shape[0]
    print(f'Before -> Rows {rows} in old category, {rows_new} in new category')

    # move to new category
    df.loc[df['product'].eq(old_product), 'product'] = new_product

    # affected rows
    rows = df[df['product'].eq(old_product)].shape[0]
    rows_new = df[df['product'].eq(new_product)].shape[0]
    print(f'After  -> Rows {rows} in old category, {rows_new} in new category')

# Moving CDs
convert_to_product('CDs', 'Savings & CDs')

# Moving Conventional
convert_to_product('Conventional', 'Credit Cards')

# Moving CD Rates
convert_to_product('CD Rates', 'Savings & CDs')

# Moving Savings
convert_to_product('Savings', 'Savings & CDs')

Before -> Rows 7 in old category, 663 in new category
After  -> Rows 0 in old category, 670 in new category
Before -> Rows 1 in old category, 17193 in new category
After  -> Rows 0 in old category, 17194 in new category
Before -> Rows 1 in old category, 670 in new category
After  -> Rows 0 in old category, 671 in new category
Before -> Rows 48 in old category, 671 in new category
After  -> Rows 0 in old category, 719 in new category


### Removing small sample products

In [13]:
# REMOVE product categories with counts less than 250 --> remaining products would be 6
df = df.groupby('product').filter(lambda x: len(x) > 250)

# See new counts of remaining products
df['product'].value_counts()

Credit Cards         17194
Checking              3965
Car Loans              758
Savings & CDs          719
Mortgages              389
Business Services      262
Name: product, dtype: int64

# Distribution Analysis

In [14]:
# Selecting small data slice for testing
SUB_SAMPLE = True

if SUB_SAMPLE:
    df = df.sample(frac=.20)

print('Data size: {} rows'.format(df.shape[0]))

Data size: 6986 rows


In [15]:
# Joining Reviews
def join_reviews(df, text_col):
    text = ''
    for _, row in df.iterrows():
        # print(row[column])
        text += row[text_col]
    return text

# 

In [16]:
# Function to return
def list_words(doc:spacy.tokens.doc.Doc, excluded:list=[], pos=None):
    if pos:
        words = [token.lemma_ for token in doc if (not token.is_punct and 
                                                   not token.is_stop and 
                                                   token.lemma_ not in excluded and
                                                   token.pos_ == pos)]    # conditional for POS type
    else:
        words = [token.lemma_ for token in doc if (not token.is_punct and 
                                                   not token.is_stop and 
                                                   token.lemma_ not in excluded)]
    return words
    

In [17]:
# TODO: convert to dictionay -> categories: text, 
#                                           words: counts

# initializing nlp
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 2_500_000

# Create dictionary
reviews = dict()

# List of unique products
products = df['product'].unique().tolist()

# Excluded words
bank_names = list({name for bank in banks for name in bank.split()})
excluded = [' ', 'bank', 'account', 'tell', 'year', 'day', '2', '$', '3', 'hold', 'know', 'go'] + bank_names

# Number of words to select
num = 10

# Join all texts by grouping
for prod in products:
    # extend excluded to exclude product mention as singular nouns (removes s) with exception business
    excluded += [word.rstrip('s') for word in prod.lower().split() if word != 'business'] 
    # adding business when that is prod
    if prod.lower() == 'business services':
        excluded += ['business']                           
    # selecting only product
    sliced = df[df['product'].eq(prod)]
    doc = nlp(join_reviews(sliced,'review')) # join & convert to spacy doc
    # Save into text dictionary variable
    reviews.update({prod:{'text':doc}})
    # Do word count (without stopwords, punct, and lemmatized)
    top_words = list_words(reviews[prod]['text'], excluded=excluded)
    top_words = Counter(top_words).most_common(num)
    as_dict = {pair[0]:pair[1] for pair in top_words}
    # Store top words
    reviews[prod]['words'] = as_dict
    

In [19]:
# Checking words make sense
for prod in products:
    print("{}: \n {}".format(prod, reviews[prod]['words']))

Credit Cards: 
 {'pay': 2305, 'good': 1958, 'reward': 1946, 'use': 1900, 'great': 1888, 'time': 1685, 'month': 1670, 'fee': 1657, 'purchase': 1615, 'like': 1598}
Mortgages: 
 {'time': 74, 'loan': 70, 'payment': 59, 'service': 59, 'customer': 58, 'month': 44, 'process': 42, 'pay': 38, 'home': 38, 'day': 37}
Checking: 
 {'fee': 946, 'customer': 933, 'service': 917, 'check': 730, 'money': 721, 'time': 621, 'charge': 543, 'deposit': 517, 'like': 431, 'good': 415}
Savings & CDs: 
 {'money': 206, 'fee': 119, 'transfer': 113, 'customer': 109, 'service': 104, 'rate': 102, 'interest': 99, 'open': 97, 'time': 90, 'save': 82}
Car Loans: 
 {'payment': 113, 'pay': 84, 'time': 70, 'customer': 63, 'rate': 60, 'service': 55, 'work': 54, 'great': 52, 'easy': 48, 'good': 48}
Business Services: 
 {'customer': 65, 'time': 33, 'close': 26, 'money': 25, 'fee': 24, 'like': 23, 'open': 22, 'branch': 22, 'cash': 20, 'phone': 20}


In [20]:
# 1. create word count chart

# Creating Dropdown menu with products
product_dropdown = widgets.Dropdown(
    options= [prod for prod in products],
    value='Credit Cards',
    description='Bank Prod:')

# Function to update bar chart
def update_plot(product:str) -> None:
    """Feeds selected product from drop down and returns top 10 words mentioned in these product reviews
        - product: selected feature from drop down
    """
    words_dict = reviews[product]['words']

    plt.figure(figsize=(20,8), num=None)
    sns.barplot(x=list(words_dict.keys()),
    y=list(words_dict.values()),)
    plt.title(f'Top 10 words for {product}')
    plt.ylabel('Counts')
    plt.xlabel('Words')

# Shows interactive plot to select feature and plot top 10 hotels
widgets.interactive(update_plot, product=product_dropdown)

interactive(children=(Dropdown(description='Bank Prod:', options=('Credit Cards', 'Mortgages', 'Checking', 'Sa…

In [33]:
# 2. create word cloud

# Creating Dropdown menu with products
product_dropdown = widgets.Dropdown(
    options= products,
    value='Credit Cards',
    description='Bank Prod:')

# function to clean words - passing dict & prod returns clean text
def clean_text(text:spacy.tokens.doc.Doc, product:str):
    # create excluded word list
    bank_names = list({name for bank in banks for name in bank.split()})
    excluded = [' ', 'bank', 'account', 'tell', 'year', 'day', '2', '$', '3', 'hold', 'know', 'go'] + bank_names
    # extend excluded to exclude product mention as singular nouns (removes s) with exception business
    excluded += [word.rstrip('s') for word in product.lower().split() if word != 'business'] 
    # adding business when that is prod
    if product.lower() == 'business services':
        excluded += ['business']   
    # get list of words and convert to clean text
    words = list_words(text, excluded=excluded)
    clean_text = ' '.join(words)
    return clean_text

def make_cloud(product:str, word_max:int):
    # get product text
    text = reviews[product]['text']
    # clean text
    cleaned_text = clean_text(text, product)
    plt.figure(figsize=(20,8))
    wordcloud = WordCloud(background_color='white', max_words=word_max).generate(cleaned_text)
    plt.style.use("dark_background")
    plt.title(f'Most used words for {product} product', fontsize=40)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")

widgets.interact(make_cloud, product=product_dropdown, word_max=[20,50,100,200])
# -> Make these interactive

interactive(children=(Dropdown(description='Bank Prod:', options=('Credit Cards', 'Mortgages', 'Checking', 'Sa…

<function __main__.make_cloud(product: str, word_max: int)>

In [18]:
# TODO: Look at ways of saving as Doc file before going to whole dataset, currently does not work from Doc to JSON https://spacy.io/usage/saving-loading
# ? is there other ways to save text corpus in Spacy as a organized fashion for fast use?
# import json
# with open('data/reviews.json', 'w') as fp:
#     json.dump(reviews, fp, indent=2)